In [3]:
%pylab
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
import pandas as pd
import seaborn as sns
path =  (os.path.abspath(os.path.join(r'.', os.pardir)))
import sys
sys.path.append(path)
from images_segmentation import otsu as ot
from images_segmentation import dicescore as dsc
from PIL import Image
import os
import os.path

Using matplotlib backend: <object object at 0x0000020CB2143800>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
# Set ground truth images from N2DH-GOWT1 dataset and N2DL-HeLa dataset to 0 and 1

def binarize(x):
    img = x.copy()

    for o in np.ndindex(img.shape):
        if img[o] > 0: 
           img[o] = 1
        else:
            img[o] = 0
    
    return img

In [5]:
# Preprocessing - Histogram stretching

def stretch(x):
    intensities = []
   
    img = x.copy()
    lower_quantile, upper_quantile = np.percentile(x, (2,  98))

    img[img < lower_quantile] = lower_quantile
    img[img > upper_quantile] = upper_quantile
    
    for i in np.ndindex(img.shape):
        intensities.append(img[i])
   
    img_max = max(intensities)
    img_min = min(intensities)
    img_stretch = (img-img_min)*(256 / (img_max-img_min))
    return img_stretch

In [6]:
# Load raw images of NIH3T3-dataset

img_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/img")
for f in os.listdir(path):
    img_NIH3T3.append(imread(os.path.join(path , f)))

# Load ground truth images of NIH3T3-dataset
 
gt_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/gt")
for f in os.listdir(path):
    gt_NIH3T3.append(imread(os.path.join(path , f)))

In [7]:
# Make a copy of all images of each dataset / use as "no preprocessed images"
copy_img_NIH3T3 = img_NIH3T3.copy()

In [8]:
# Apply median filter to all images of each dataset
# program das auf alle bilder die filter in utnersch. größe anwendet...programmieren


#! not filter size : t = (((w - 1)/2)-0.5)/s -> stimmt doch nicht schau nomal inet
# set window size w = 5
# synthax: filtering: filtered_data = scipy.ndimage.filters.gaussian_filter(data, sigma=s, truncate=t)
filter = [3, 4, 5, 10, 15, 20]
median_img_NIH3T3 = []
for j in range (0, len(filter)):
    
    for i in range(0, len(copy_img_NIH3T3)):
        median_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , filter[j])
    

#oder einfach andere built in function nehmen?!:https://www.geeksforgeeks.org/apply-a-gauss-filter-to-an-image-with-python/
    

SyntaxError: incomplete input (299498429.py, line 17)

In [ ]:
# Apply gaussian filter to all images of each dataset

gauss_img_NIH3T3 = []
for i in range(0,len(copy_img_NIH3T3)):
    gauss_img_NIH3T3.append(scipy.ndimage.gaussian_filter(copy_img_NIH3T3[i] , 5))

In [ ]:
# Apply histogram stretching to all images of each dataset
stretch_img_NIH3T3 = []
for i in range(0,len(copy_img_NIH3T3)):
    stretch_img_NIH3T3.append(stretch(copy_img_NIH3T3[i]))


In [ ]:
# Apply median filter and histogram strechting to all images of each dataset
median_stretch_img_NIH3T3 = []
for i in range(0,len(median_img_NIH3T3)):
    median_stretch_img_NIH3T3.append(stretch(median_img_NIH3T3[i]))

In [ ]:
# Apply histogram stretching and median filter to all images of each dataset
stretch_median_img_NIH3T3 = []
for i in range(0,len(stretch_img_NIH3T3)):
    stretch_median_img_NIH3T3.append(scipy.ndimage.median_filter(stretch_img_NIH3T3[i] , 5))

In [ ]:
# Apply gaussian filter and histogram strechting to all images of each dataset

gauss_stretch_img_NIH3T3 = []
for i in range(0,len(gauss_img_NIH3T3)):
    gauss_stretch_img_NIH3T3.append(stretch(gauss_img_NIH3T3[i]))


In [ ]:
# Apply histogram stretching and gaussian filter to all images of each dataset

stretch_gauss_img_NIH3T3 = []
for i in range(0,len(stretch_img_NIH3T3)):
    stretch_gauss_img_NIH3T3.append(scipy.ndimage.gaussian_filter(stretch_img_NIH3T3[i] , 5))

## Otsu thresholding

In [ ]:
# Apply otsu thresholding to all images of each dataset - no preprocessing
otsu_img_NIH3T3 = []
for i in range(0,len(copy_img_NIH3T3)):
    otsu_img_NIH3T3.append(ot.otsu_thresholding(copy_img_NIH3T3[i] , 256))

MemoryError: Unable to allocate 5.25 MiB for an array with shape (1376256,) and data type float32

In [ ]:
# Apply otsu thresholding to all images of each dataset - median filter

otsu_median_img_NIH3T3 = []
for i in range(0,len(median_img_NIH3T3)):
    otsu_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3[i] , 256))


In [ ]:
# Apply otsu thresholding to all images of each dataset - gaussian filter
otsu_gauss_img_NIH3T3 = []
for i in range(0,len(gauss_img_NIH3T3)):
    otsu_gauss_img_NIH3T3.append(ot.otsu_thresholding(gauss_img_NIH3T3[i] , 256))


## Dice score

In [ ]:
# Dice score - N2DH-GOWT1-dataset
## median filter

dice_otsu_median_img_N2DH_GOWT1 = []
for j in range(0,len(otsu_median_img_N2DH_GOWT1)):
    dice_otsu_median_img_N2DH_GOWT1.append(dsc.dice(otsu_median_img_N2DH_GOWT1[j] , binary_gt_N2DH_GOWT1[j]))

print(dice_otsu_median_img_N2DH_GOWT1)


NameError: name 'otsu_median_img_N2DH_GOWT1' is not defined

In [ ]:
# Dice score - N2DH-GOWT1-dataset
## gaussian filter

dice_otsu_gauss_img_N2DH_GOWT1 = []
for j in range(0,len(otsu_gauss_img_N2DH_GOWT1)):
    dice_otsu_gauss_img_N2DH_GOWT1.append(dsc.dice(otsu_gauss_img_N2DH_GOWT1[j] , binary_gt_N2DH_GOWT1[j]))

print(dice_otsu_gauss_img_N2DH_GOWT1)

NameError: name 'otsu_gauss_img_N2DH_GOWT1' is not defined